<a href="https://colab.research.google.com/github/TinaKhatri28/ASTRA-Project/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content"

drive  sample_data


In [ ]:
!ls "/content/drive/MyDrive/sih project/model2 data"

 Cashew   Cassava  'Cotton leaves'   Maize   Tomato


In [ ]:
BASE_PATH = "/content/drive/MyDrive/sih project/model2 data/Cotton leaves/40 Images"

In [ ]:
import os
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def get_disease_classes(path):
    return sorted([
        folder for folder in os.listdir(path)
        if os.path.isdir(os.path.join(path, folder))
    ])

CLASSES = get_disease_classes(BASE_PATH)

print("✅ TOTAL DISEASE CLASSES FOUND:", len(CLASSES))
print("✅ Classes:", CLASSES)

GLOBAL_CLASSES = sorted([
    folder for folder in os.listdir(BASE_PATH)
    if os.path.isdir(os.path.join(BASE_PATH, folder))
])

print("✅ CLASSES FOUND:", GLOBAL_CLASSES)

✅ TOTAL DISEASE CLASSES FOUND: 6
✅ Classes: ['Aphids', 'Army worm', 'Bacterial blight', 'Powdery mildew', 'Target spot', 'txt images']
✅ CLASSES FOUND: ['Aphids', 'Army worm', 'Bacterial blight', 'Powdery mildew', 'Target spot', 'txt images']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

#  Train generator
train_generator = train_datagen.flow_from_directory(
    BASE_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=GLOBAL_CLASSES,
    subset='training',
    shuffle=True
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    BASE_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=GLOBAL_CLASSES,
    subset='validation',
    shuffle=False
)


Found 1707 images belonging to 6 classes.
Found 424 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(GLOBAL_CLASSES), activation='softmax')
])

model.compile(
    optimizer=Adam(0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,587,462 (9.87 MB)

 Trainable params: 329,478 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint("model2_best.h5", save_best_only=True, monitor="val_accuracy", mode="max"),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3)
]


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    epochs=20,
    callbacks=callbacks
)




/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.4443 - loss: 1.4566

53/53 ━━━━━━━━━━━━━━━━━━━━ 548s 10s/step - accuracy: 0.4470 - loss: 1.4496 - val_accuracy: 0.8077 - val_loss: 0.5082 - learning_rate: 5.0000e-04
Epoch 2/20
 1/53 ━━━━━━━━━━━━━━━━━━━━ 1:07 1s/step - accuracy: 0.6875 - loss: 0.6123

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


53/53 ━━━━━━━━━━━━━━━━━━━━ 24s 442ms/step - accuracy: 0.6875 - loss: 0.6123 - val_accuracy: 0.8149 - val_loss: 0.5088 - learning_rate: 5.0000e-04
Epoch 3/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7603 - loss: 0.6232

53/53 ━━━━━━━━━━━━━━━━━━━━ 145s 3s/step - accuracy: 0.7608 - loss: 0.6221 - val_accuracy: 0.9038 - val_loss: 0.2967 - learning_rate: 5.0000e-04
Epoch 4/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 40s 719ms/step - accuracy: 0.9062 - loss: 0.4274 - val_accuracy: 0.9014 - val_loss: 0.2939 - learning_rate: 5.0000e-04
Epoch 5/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8268 - loss: 0.4739

53/53 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - accuracy: 0.8267 - loss: 0.4740 - val_accuracy: 0.9111 - val_loss: 0.2559 - learning_rate: 5.0000e-04
Epoch 6/20
 1/53 ━━━━━━━━━━━━━━━━━━━━ 1:09 1s/step - accuracy: 0.8438 - loss: 0.3905

53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 469ms/step - accuracy: 0.8438 - loss: 0.3905 - val_accuracy: 0.9183 - val_loss: 0.2518 - learning_rate: 5.0000e-04
Epoch 7/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8634 - loss: 0.3871

53/53 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.8633 - loss: 0.3870 - val_accuracy: 0.9519 - val_loss: 0.1923 - learning_rate: 5.0000e-04
Epoch 8/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 42s 777ms/step - accuracy: 0.7500 - loss: 0.6323 - val_accuracy: 0.9399 - val_loss: 0.1926 - learning_rate: 5.0000e-04
Epoch 9/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.8807 - loss: 0.3337 - val_accuracy: 0.9303 - val_loss: 0.2469 - learning_rate: 5.0000e-04
Epoch 10/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 759ms/step - accuracy: 0.8750 - loss: 0.4584 - val_accuracy: 0.9327 - val_loss: 0.2422 - learning_rate: 5.0000e-04
Epoch 11/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 148s 3s/step - accuracy: 0.8745 - loss: 0.3455 - val_accuracy: 0.9495 - val_loss: 0.1714 - learning_rate: 1.5000e-04
Epoch 12/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 454ms/step - accuracy: 0.8750 - loss: 0.3302 - val_accuracy: 0.9495 - val_loss: 0.1709 - learning_rate: 1.5000e-04
Epoch 13/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.8794 -

53/53 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.9026 - loss: 0.2732 - val_accuracy: 0.9543 - val_loss: 0.1598 - learning_rate: 1.5000e-04
Epoch 16/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 42s 775ms/step - accuracy: 0.8750 - loss: 0.3519 - val_accuracy: 0.9519 - val_loss: 0.1580 - learning_rate: 1.5000e-04
Epoch 17/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 161s 3s/step - accuracy: 0.9014 - loss: 0.2590 - val_accuracy: 0.9351 - val_loss: 0.1921 - learning_rate: 1.5000e-04
Epoch 18/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 25s 448ms/step - accuracy: 0.9062 - loss: 0.2168 - val_accuracy: 0.9351 - val_loss: 0.1941 - learning_rate: 1.5000e-04
Epoch 19/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.9075 - loss: 0.2517 - val_accuracy: 0.9399 - val_loss: 0.1857 - learning_rate: 1.5000e-04
Epoch 20/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 785ms/step - accuracy: 0.9091 - loss: 0.1669 - val_accuracy: 0.9375 - val_loss: 0.1881 - learning_rate: 4.5000e-05


In [ ]:
model.save("/content/drive/MyDrive/sih_project/model2_final_cotton_disease.h5")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_disease(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_id = np.argmax(preds)
    confidence = preds[0][class_id]

    return GLOBAL_CLASSES[class_id], confidence

# Example
path = "/content/drive/MyDrive/sih project/model2 data/Cotton leaves/40 Images/Aphids/6.jpg"
label, conf = predict_disease(path)
print("Predicted:", label, "Confidence:", conf)



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: Aphids Confidence: 0.99954706
